In [ ]:
!pip install scikit-learn==0.24

     |████████████████████████████████| 22.3 MB 4.4 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1


In [ ]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score,explained_variance_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.cm as cm
import re

In [ ]:
datosOPF = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Operacion de Sistemas ML/dataset.csv',header=None)
datosOPF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235
0,55.347,19.178,24.461,35.694,0,39.316,22.728,25.232,0,0,74.115,42.567,37.407,17.7130,74.707,12.523,9.1530,76.826,47.188,22.915,17.767,12.1660,4.4094,8.6472,0,0,62.417,16.993,35.369,0,45.641,47.502,23.509,29.927,43.058,34.294,0,0,27.616,50.506,...,39.792,38.928,38.781,40.063,39.924,39.348,38.928,39.318,39.240,38.520,37.782,40.151,39.784,38.907,39.436,39.604,39.923,39.782,39.420,39.365,39.496,39.156,39.408,39.115,39.647,40.231,40.491,40.799,40.974,40.489,40.471,40.292,39.731,40.801,39.829,40.439,40.447,38.778,40.262,40.718
1,68.257,23.043,37.473,56.098,0,47.882,16.438,37.758,0,0,59.065,50.125,37.141,8.3523,47.471,23.282,7.5745,48.866,50.775,17.355,18.496,8.9759,5.0229,16.8120,0,0,45.114,16.901,13.643,0,52.958,75.355,28.107,82.770,35.696,33.996,0,0,31.471,45.734,...,39.113,38.313,38.220,39.873,39.799,39.360,38.886,39.141,39.064,38.525,37.691,40.022,39.641,38.736,39.241,39.455,39.809,39.545,39.047,38.972,39.270,38.955,39.126,38.903,39.505,40.122,40.309,40.442,40.864,40.369,40.383,40.264,39.701,40.990,39.790,40.390,40.383,38.249,40.851,40.698
2,49.366,26.603,52.136,47.119,0,44.639,12.473,34.415,0,0,55.451,56.814,36.982,16.3610,132.280,30.754,14.2240,52.753,52.281,23.221,13.363,7.2361,6.9883,14.4850,0,0,37.055,11.935,16.488,0,29.415,67.432,14.890,62.589,47.405,26.936,0,0,15.806,82.389,...,39.414,38.555,38.485,39.200,38.871,37.940,37.307,37.560,37.485,36.770,36.153,37.673,37.627,37.378,38.197,38.675,39.018,39.112,38.922,38.789,39.099,38.535,38.432,37.760,39.306,40.114,40.323,40.467,40.573,40.388,40.401,40.236,39.671,40.595,39.996,40.401,40.405,38.544,41.257,40.828
3,76.195,13.548,54.495,31.687,0,40.881,10.135,31.870,0,0,84.320,24.928,18.288,17.9780,58.272,33.523,6.5199,71.553,45.249,13.614,16.416,12.2090,4.9861,7.8468,0,0,89.310,22.570,35.215,0,47.045,69.068,18.924,62.767,25.206,38.662,0,0,37.136,34.476,...,39.088,38.120,38.070,39.221,39.020,38.387,37.752,38.128,38.056,37.252,36.274,37.367,37.541,37.425,38.119,38.588,38.998,38.938,38.604,38.329,38.498,38.178,38.185,37.703,38.785,39.436,39.680,39.790,40.563,39.777,39.805,39.705,39.165,40.291,39.799,40.596,40.599,38.089,40.789,40.881
4,54.804,19.101,45.123,28.032,0,65.770,12.568,25.322,0,0,100.840,23.781,31.325,9.9082,89.796,21.032,10.9340,76.157,55.210,22.740,11.354,14.7390,5.6986,7.0608,0,0,105.000,23.051,23.242,0,30.618,47.354,14.538,74.457,35.510,28.232,0,0,39.102,66.368,...,38.621,37.760,37.698,38.553,38.267,37.682,37.170,37.426,37.358,36.828,36.092,37.954,37.741,37.075,37.732,38.165,38.494,38.384,38.136,38.248,38.419,38.018,37.900,37.375,38.867,39.701,39.996,40.145,40.432,40.183,40.235,40.185,39.619,40.786,39.792,40.480,40.499,37.696,40.389,40.293


In [ ]:
datosOPF.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.00000,1000.000000,1000.000000,1000.0,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.0,1000.0,1000.000000,1000.000000,1000.000000,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.0,1000.0,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,51.025123,19.904502,39.481376,38.715308,0.0,51.56094,18.768560,27.579996,0.0,0.0,70.562549,47.101672,34.387103,14.101968,89.093061,25.337621,11.041586,60.925173,44.825629,17.914872,13.92384,10.108888,6.980736,12.971888,0.0,0.0,70.606942,16.892982,23.526586,0.0,43.248002,59.547820,22.860234,59.046977,33.128242,31.204462,0.0,0.0,26.999478,66.406312,...,38.905347,38.068064,37.986783,39.125586,38.902037,38.298758,37.780363,38.218005,38.145581,37.313776,36.54261,38.319910,38.144033,37.611876,38.267860,38.648727,38.992075,38.897172,38.583029,38.524928,38.641668,38.342408,38.298242,37.867515,39.094084,39.834192,40.054052,40.174317,40.549563,40.166753,40.196210,40.108032,39.548118,40.707573,39.771368,40.398245,40.404326,37.994733,40.668140,40.419831
std,14.775594,5.661968,11.119501,10.963833,0.0,14.75276,5.526369,8.087310,0.0,0.0,20.224814,13.043657,9.831846,4.041932,26.039669,7.060879,3.128883,17.319339,13.185947,5.261204,4.07312,2.850939,2.035195,3.826526,0.0,0.0,19.991753,4.932775,6.938531,0.0,12.253502,17.096811,6.747260,17.080344,9.584097,8.750951,0.0,0.0,7.830875,19.588404,...,0.388902,0.357692,0.335070,0.456908,0.486231,0.557562,0.593612,0.635262,0.632293,0.635964,0.62259,1.041322,0.856883,0.588939,0.501738,0.430981,0.442520,0.416298,0.379632,0.361687,0.376284,0.341072,0.453497,0.541530,0.275333,0.275551,0.252244,0.293284,0.235743,0.218367,0.210675,0.200028,0.191775,0.256293,0.157224,0.234135,0.237643,0.336445,0.319807,0.311227
min,25.502000,10.009000,19.514000,19.592000,0.0,26.00200,9.512000,14.052000,0.0,0.0,35.162000,23.554000,17.031000,7.020100,45.162000,12.523000,5.502400,30.091000,22.540000,9.004400,7.03200,5.029900,3.502300,6.503500,0.0,0.0,35.586000,8.504800,12.008000,0.0,21.517000,29.539000,11.509000,29.643000,16.536000,15.507000,0.0,0.0,13.514000,33.172000,...,37.348000,36.733000,36.731000,37.404000,37.179000,36.594000,35.989000,36.314000,36.245000,35.425000,34.81300,35.781000,35.899000,35.893000,36.538000,36.992000,37.234000,37.219000,37.108000,37.131000,37.192000,36.993000,36.892000,36.247000,37.991000,38.585000,38.808000,38.852000,39.343000,39.155000,39.233000,39.200000,38.674000,40.016000,38.783000,39.053000,39.043000,36.787000,39.446000,38.575000
25%,38.700250,15.175500,30.355500,29.318500,0.0,38.68425,13.795750,20.682000,0.0,0.0,52.967500,36.001500,25.892500,10.595500,67.595000,19.254750,8.423525,45.675000,33.642500,13.310750,10.31600,7.782925,5.168250,9.817100,0.0,0.0,54.158250,12.664250,17.442750,0.0,32.218250,44.490750,16.833500,44.614250,24.732500,24.123000,0.0,0.0,20.624250,48.820000,...,38.666750,37.855250,37.781750,38.828750,38.573250,37.919750,37.366750,37.776000,37.707000,36.867000,36.09175,37.493250,37.482750,37.208750,37.944500,38.375000,38.701500,38.629750,38.339000,38.299000,38.406000,38.130750,38.004750,37.506750,3

In [ ]:
X = datosOPF.iloc[:,0:118].to_numpy()
Y = datosOPF.iloc[:,118:236].to_numpy()
print("X:", X.shape, "Y:", Y.shape)

X: (1000, 118) Y: (1000, 118)


In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, Y, test_size=0.15, random_state=42)
print("X train:", X_train.shape, "X test:", X_test.shape)
print("Y train:", y_train.shape, "Y test:", y_test.shape)

X train: (850, 118) X test: (150, 118)
Y train: (850, 118) Y test: (150, 118)


#Grafica de variación de Cargas

In [ ]:
colormap =cm.get_cmap("gist_rainbow") #choose any matplotlib colormap here
colorspan=[40,140]
N = 4000
a = np.random.random(size=N) * 100
b = np.random.random(size=N) * 100
cmap_input=np.interp(np.sqrt(a*a+b*b),colorspan,[0,1],left=0,right=1)
A_color=colormap(cmap_input,1,True)
bokeh_colors = ["#%02x%02x%02x" % (r, g, b) for r, g, b in A_color[:,0:3]]

In [ ]:
figLMC = go.Figure()
for i in range(0,X_test.shape[1]):
  figLMC.add_trace(go.Scatter(y=y_test[:,i],
                    mode='lines',
                    line=dict(width=1)))

figLMC.update_xaxes(title_text='<b>Iteration</b>',titlefont_size=22,tickfont_size=16, tickmode = 'linear', tick0 = 0, dtick = 10)
figLMC.update_yaxes(title_text='<b>LMP ($/kWh)</b>',titlefont_size=22,tickfont_size=16)


figLMC.update_layout(width=1500,
                     height=750,
                     paper_bgcolor='rgba(0,0,0,0)',
                     plot_bgcolor='rgba(0,0,0,0)',
                     yaxis=dict(color="black"),
                     xaxis=dict(color="black"),
                     showlegend=False
                    ) 
figLMC.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
figLMC.update_yaxes(showline=True, linewidth=2, gridcolor='lightgray', linecolor='black', mirror=False)
figLMC.show(renderer="colab")

In [ ]:
x_scaler = MinMaxScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

y_scaler = MinMaxScaler()
Y_train_scaled = y_scaler.fit_transform(y_train)
Y_test_scaled = y_scaler.transform(y_test)

# SVR

## Ajuste de Hiperparametros

In [ ]:
param_grid = {'estimator__C':[0.1, 0.5, 1, 10],
              'estimator__epsilon':[0.1, 0.2, 0.3, 0.4]}

svr = SVR()
mor = RandomizedSearchCV(MultiOutputRegressor(svr), n_jobs=-1, cv=3, param_distributions=param_grid, n_iter=5, random_state=42, scoring='neg_mean_absolute_error')
mor.fit(X_train, y_train)
mor.best_params_, mor.best_score_

({'estimator__C': 10, 'estimator__epsilon': 0.2}, -0.10391978859507944)

In [ ]:
svrO= SVR(epsilon=mor.best_params_["estimator__epsilon"],C=mor.best_params_["estimator__C"])
mulout = MultiOutputRegressor(svrO)
mulout.fit(X_train,y_train)
y_pred = mulout.predict(X_test)

In [ ]:
aux = range(0, y_pred.shape[0])
y_pred_train = mulout.predict(X_train)

## Feature Importance

# Datos de prueba y predicion de un solo nodo

In [ ]:
figLMP1B = make_subplots(rows=5, cols=2, subplot_titles=['<b>Bus 1</b>', '<b>Bus 15</b>', '<b>Bus 22</b>', '<b>Bus 35</b>', '<b>Bus 53</b>',
                                                         '<b>Bus 66</b>', '<b>Bus 90</b>', '<b>Bus 105</b>','<b>Bus 111</b>', '<b>Bus 118</b>']
                         , vertical_spacing=0.06, horizontal_spacing=0.08)

figLMP1B.add_trace(go.Scatter(y=y_test[:,0],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=1, col=1)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,0],
                    mode='lines',
                    line=dict(width=2, color="red")), row=1, col=1)

figLMP1B.add_trace(go.Scatter(y=y_test[:,14],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=1, col=2)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,14],
                    mode='lines',
                    line=dict(width=2, color="red")), row=1, col=2)


figLMP1B.add_trace(go.Scatter(y=y_test[:,21],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=2, col=1)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,21],
                    mode='lines',
                    line=dict(width=2, color="red")), row=2, col=1)

figLMP1B.add_trace(go.Scatter(y=y_test[:,34],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=2, col=2)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,34],
                    mode='lines',
                    line=dict(width=2, color="red")), row=2, col=2)


figLMP1B.add_trace(go.Scatter(y=y_test[:,52],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=3, col=1)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,52],
                    mode='lines',
                    line=dict(width=2, color="red")), row=3, col=1)
figLMP1B.add_trace(go.Scatter(y=y_test[:,65],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=3, col=2)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,65],
                    mode='lines',
                    line=dict(width=2, color="red")), row=3, col=2)


figLMP1B.add_trace(go.Scatter(y=y_test[:,89],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=4, col=1)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,89],
                    mode='lines',
                    line=dict(width=2, color="red")), row=4, col=1)

figLMP1B.add_trace(go.Scatter(y=y_test[:,104],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=4, col=2)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,104],
                    mode='lines',
                    line=dict(width=2, color="red")), row=4, col=2)


figLMP1B.add_trace(go.Scatter(y=y_test[:,110],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=5, col=1)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,110],
                    mode='lines',
                    line=dict(width=2, color="red")), row=5, col=1)

figLMP1B.add_trace(go.Scatter(y=y_test[:,117],
                    mode='lines',
                    line=dict(width=2, color="blue")), row=5, col=2)
figLMP1B.add_trace(go.Scatter(y=y_pred[:,117],
                    mode='lines',
                    line=dict(width=2, color="red")), row=5, col=2)


figLMP1B.update_layout(width=4000,
                       height=5000,
                       paper_bgcolor='rgba(0,0,0,0)',
                       plot_bgcolor='rgba(0,0,0,0)',
                       showlegend=False,
                       font=dict(size=35,
                               color="Black"),
                       ) 
figLMP1B.update_xaxes(tickmode = 'linear', tick0 = 0, dtick = 10, color="black", showline=True, linewidth=2, linecolor='black', mirror=False, 
                      title='<b>Iterations</b>',titlefont_size=50)
figLMP1B.update_yaxes(color="black", showline=True, linewidth=2, gridcolor='lightgray', linecolor='black', mirror=False, title='<b>LMP ($USD/kWh)</b>',
                       titlefont_size=50)
figLMP1B.update_annotations(dict(font_size=65, borderpad=5, xshift=-30))
figLMP1B.show(renderer="colab")

# Metricas de Regresión

In [ ]:
print("MSE:",mean_squared_error(y_test,y_pred))
print("RMSE:",math.sqrt(mean_squared_error(y_test,y_pred)))
print("MAE:",mean_absolute_error(y_test,y_pred))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred)*100)
print("R2:", r2_score(y_test,y_pred))
print("Explained Variance:", explained_variance_score(y_test,y_pred))

MSE: 0.013832662442829631
RMSE: 0.11761233967075747
MAE: 0.0943801315270787
MAPE: 0.2391214393206476
R2: 0.8217823392332984
Explained Variance: 0.8430556511892288


In [ ]:
print("MSE:",mean_squared_error(y_train,y_pred_train))
print("RMSE:",math.sqrt(mean_squared_error(y_train,y_pred_train)))
print("MAE:",mean_absolute_error(y_train,y_pred_train))
print("MAPE:",mean_absolute_percentage_error(y_train,y_pred_train)*100)
print("R2:", r2_score(y_train,y_pred_train))
print("Explained Variance:", explained_variance_score(y_train,y_pred_train))

MSE: 0.012086175191346088
RMSE: 0.10993714200099113
MAE: 0.09148555060012113
MAPE: 0.23155846236502506
R2: 0.8283751680728685
Explained Variance: 0.8578363049345403


In [ ]:
figLMP = go.Figure()
figLMP.add_trace(go.Bar(name="Data",y=y_test[1,:],x=list(range(0,118)), marker_color="blue"))
figLMP.add_trace(go.Bar(name="Forecast",y=y_pred[1,:],x=list(range(0,118)), marker_color="red"))

figLMP.update_layout(barmode='group',
                  width=1400,
                  height=720,
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  yaxis=dict(
                    title='<b>LMP ($USD/kWh)</b>',
                    titlefont_size=22,
                    tickfont_size=16,
                    color="black",
                    range=[36.5, 42]
                  ),
                  xaxis=dict(
                    title='<b>Buses</b>',
                    titlefont_size=22,
                    tickfont_size=16,
                    color="black",
                    tickangle=270
                  ),
                  legend=dict(
                    x=0,
                    y=1.0,
                    font = dict(size = 18, color = "black")
                  ))
figLMP.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
figLMP.update_yaxes(showline=True, linewidth=2, gridcolor='lightgray', linecolor='black', mirror=False)
figLMP.show(renderer="colab")

In [ ]:
error = ((y_pred - y_test)/y_test)*100

In [ ]:
values = error[error>0]
len(values)

7081

In [ ]:
figerror = go.Figure()
figerror.add_trace(go.Bar(name="Error",y=error[20,:],x=list(range(0,118)), marker_color="blue"))

figerror.update_layout(barmode='group',
                  width=1400,
                  height=720,
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  yaxis=dict(
                    title='<b>Relative Forecasting Error (%) </b>',
                    titlefont_size=22,
                    tickfont_size=16,
                    color="black"
                  ),
                  xaxis=dict(
                    title='<b>Buses</b>',
                    titlefont_size=22,
                    tickfont_size=16,
                    color="black"
                  ),
                  legend=dict(
                    x=0,
                    y=1.0,
                    font = dict(size = 18, color = "black")
                  ))
figerror.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=False)
figerror.update_yaxes(showline=True, linewidth=2, gridcolor='lightgray', linecolor='black', mirror=False)
figerror.show(renderer="colab")

Caso Adicional

In [ ]:
datOPFind = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Operacion de Sistemas ML/datasetInd.csv',header=None)
datOPFind.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235
0,31.526,29.768,40.559,47.07,0,40.968,17.376,27.016,0,0,42.016,31.871,29.228,7.7939,91.97,20.896,7.4324,42.537,63.232,21.157,13.559,14.121,4.2281,16.192,0,0,48.578,18.653,19.198,0,30.642,82.302,13.143,43.807,18.274,29.193,0,0,18.81,39.162,...,39.002,38.08,38.013,39.329,39.21,38.851,38.349,38.907,38.831,37.792,37.15,39.439,39.104,38.196,38.645,38.916,39.09,38.986,38.648,38.585,38.925,38.65,38.75,38.416,39.356,40.152,40.358,40.535,40.516,40.441,40.461,40.368,39.797,41.04,39.477,40.107,40.096,38.068,39.89,40.445


In [ ]:
X = datOPFind.iloc[:,0:118].to_numpy()
Y = datOPFind.iloc[:,118:236].to_numpy()
print("X:", X.shape, "Y:", Y.shape)

X: (1, 118) Y: (1, 118)


In [ ]:
y_pred_ind = mulout.predict(X)

In [ ]:
figLMP2 = go.Figure()
figLMP2.add_trace(go.Bar(name="Datos Reales",y=Y[0,:],x=list(range(0,118)), marker_color="blue"))
figLMP2.add_trace(go.Bar(name="Predicción",y=y_pred_ind[0,:],x=list(range(0,118)), marker_color="red"))

figLMP2.update_layout(barmode='group',
                  width=1400,
                  height=720,
                  yaxis=dict(
                    title='<b>LMP ($/kWh)</b>',
                    titlefont_size=22,
                    tickfont_size=16,
                    range=[36.5, 42]
                  ),
                  xaxis=dict(
                    title='<b>Buses</b>',
                    titlefont_size=22,
                    tickfont_size=16,
                    #tickangle=270
                  ),
                  legend=dict(
                    x=0,
                    y=1.0,
                    font = dict(size = 18, color = "black")
                  ))
figLMP2.show(renderer="colab")